In [7]:
using Plots # or StatsPlots
using LinearAlgebra
using SpecialFunctions
using AssociatedLegendrePolynomials
using LaTeXStrings
using Quantikz
using SparseArrays # https://docs.julialang.org/en/v1/stdlib/SparseArrays/


In [11]:

# import conventions
include("conventions.jl")
  using .conventions: little_endian, big_endian, qubit_begin, err_tol, show_op_mat, show_qc_mat

  # import quantum gates
include("quantum_gates.jl")
using ..quantum_gates: q, Rz_gate1

include("lib_tensor/QTensor.jl")
using ..QTensor: q_T2D, q_T4D


include("lib_useful/custom_functions.jl")
using ..custom_functions: MK_sortrows

include("quantum_circuit.jl")
using ..quantum_circuit: qc_init, init_register, show_statevector, op
#using ..quantum_circuit: qc_init, init_register, print_initstate

Load quantum gates constructor


Load Tensor module: QTensor.jl
Load quantum gates constructor
Load quantum_circuit constructor
Load quantum_circuit constructor


Load quantum gates constructor
Load quantum_circuit constructor


In [2]:
Base.@kwdef mutable struct qc_initstruct1
    # Initialize the quantum register
    n_qubits::Int64
    q_order::String
    n_bas::Int64
    n_dim::Int64
    state_vector
#    state_vector::Array{Float64,1}
    q_states
# table of quantum gates and the operations on the quantum register    
    op_table::Matrix
# Store the quantum circuit matrix representation
    qc_matrix
# big_endian is used to enforce a default convention 
    big_endian::Bool 
# show_matrix is used to print the matrix or the quantum gates by default    
    show_op_mat::Bool
# show the matrix representation of the quantum circuit
    show_qc_mat::Bool
#    q_states::Array{Float64,2}
end # end qc_initialize


qc_initstruct1

In [17]:
# initialize a table with first row of [action gate/operator control1_qubit,   control2_qubit, target_qubit theta     phi     lambda; gate_object]
function init_op_tab()
    # initialize the first row of the table q_tab 
    # Op_ind is the index of the operation in the quantum circuit
    # Op is the name of the operation
    # q1_control is the first control qubit
    # q2_control is the second control qubit
    # q_target is the target qubit
    # theta is the angle of rotation theta 
    # phi is the angle of rotation phi
    # lambda is the angle of rotation lambda
    # object is the object of the operation, could be a gate function or a matrix
    # op_mat is the matrix representation of the operation
#    op_tab[2,1:9]=["init" "init" 0 0 0 0 0 0 0]
    op_tab = Matrix(undef, 1,10)
    op_tab[1,1:10]=["Op_ind","Op","q1_control","q2_control","q_target", 
                "theta","phi","lambda","object","op_mat"]
    return op_tab
end # end init_op_tab


init_op_tab (generic function with 1 method)

In [18]:
op_tab=init_op_tab()

1×10 Matrix{Any}:
 "Op_ind"  "Op"  "q1_control"  …  "phi"  "lambda"  "object"  "op_mat"

In [38]:


function qc_init_test(n::Int64;
    big_endian::Bool=conventions.big_endian,
    c_sv= nothing, 
#     c_sv::Union{Vector{Float64}, Vector{Int64}, Vector{ComplexF64}} = nothing,     
    err_tol::Float64=conventions.err_tol,
    show_op_mat::Bool=conventions.show_op_mat,
    show_qc_mat::Bool=conventions.show_qc_mat)
    # Initialize the quantum register
    # n::Int64: number of qubits
    # q_order::String: order of the qubits in the quantum register
    # return: quantum register of n qubits
    # q_order == "big-endian"
    # q_order == "little-endian"

    # start the function
    n_bas = 2 # number of basis states
    n_qubits = n # number of qubits
    n_dim = 2^n # dimensions of the quantum register/Hilbert space
    state_vector = zeros(2^n) # initialize the state_vector
    # create the default statevector of the quantum register: 
    state_vector[1] = 1 # set the initial state to |000 ...0>
    q_states = zeros(2^n, n) # initialize the quantum states
    q_tab = [0;1] # initialize the basis vectors quantum table
   
    # store the order of the qubits, big-endian or little-endian
    # for printing purpose only 
    if big_endian
        q_order = "big-endian"
    else
        q_order = "little-endian"
    end

    ## notes:
    # the minimum number of qubits is 2 
    if n_qubits == 1
        q_states = q_tab # basis vectors
        # check if the user has provided a custom statevector
        # err_tol = 1e-16 # error tolerance for checking the unitary condition
        if c_sv != nothing
            # check if the custom statevector has the correct dimensions
            if length(c_sv) != n_dim
                error("The custom statevector has the wrong dimensions")
            end
            if isapprox(norm(c_sv), 1, rtol=err_tol) == false
                error("The custom statevector is not normalized")
            end
            state_vector = c_sv
        end 
   else
        n_count = n_bas
        if !big_endian
            for i = 2:n_qubits
                q_tab = [ q_tab zeros(n_count,1)
                       q_tab  ones(n_count,1) ]
                n_count = n_count*2
            end
        else
            for i = 2:n_qubits
                q_tab = [zeros(n_count,1) q_tab
                        ones(n_count,1) q_tab]
                n_count = n_count*2
            end
        end
        # for i = 2:n_qubits
        #     q_tab = [zeros(n_count,1) q_tab
        #             ones(n_count,1) q_tab]
        #     n_count = n_count*2
        # end
        q_states = q_tab # basis vectors 
    end # end if n_qubits == 1

    # check if the user has provided a custom statevector
    # err_tol = 1e-16 # error tolerance for checking the unitary condition
    if c_sv != nothing
        # check if the custom statevector has the correct dimensions
        if length(c_sv) != n_dim
            error("The custom statevector has the wrong dimensions")
        end
        if isapprox(norm(c_sv), 1, rtol=err_tol) == false
            error("The custom statevector is not normalized")
        end
        state_vector = c_sv
    end
    # one final check for the statevector
    if isapprox(norm(state_vector), 1, rtol=err_tol) == false
        error("The statevector is not normalized")
    end
    # Initialize the matrix representation of the quantum circuit to 
    # the identity matrix of size n_dim
    qc_matrix = Matrix(I, n_dim, n_dim)

    # initiate the table of quantum gates and operations
    op_table = init_op_tab()
#    return qc_initstruct(n_qubits, q_order, n_bas, n_dim, 1.0, [1.0 1.0])
    return qc_initstruct1(n_qubits, q_order, n_bas, n_dim, state_vector, 
                        q_states, op_table, qc_matrix,big_endian,show_op_mat,show_qc_mat)

#    return 1

end # end qc_initialize  


qc_init_test (generic function with 1 method)

In [20]:
qc_matrix = Matrix(I, 2, 2)


2×2 Matrix{Bool}:
 1  0
 0  1

In [39]:
qc = qc_init_test(2)

qc_initstruct1(2, "big-endian", 2, 4, [1.0, 0.0, 0.0, 0.0], [0.0 0.0; 0.0 1.0; 1.0 0.0; 1.0 1.0], Any["Op_ind" "Op" … "object" "op_mat"], Bool[1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 1], true, false, false)

In [22]:
qc.op_table

1×10 Matrix{Any}:
 "Op_ind"  "Op"  "q1_control"  …  "phi"  "lambda"  "object"  "op_mat"

In [23]:
# prepare the gate in case of being parametrized, i.e. rotational gate 
# or a phase gate
function qU_prep(Qgate;
    theta=nothing,phi=nothing,lambda =nothing)
# some functions from the quantum_gates.jl file are 
# are not or this function is not adapted to. 
# We have to check them case by case 
if theta == nothing && phi == nothing && lambda == nothing
return Qgate

elseif theta != nothing && phi != nothing && lambda != nothing
Qgate = Qgate(theta=theta,phi=phi,lambda=lambda)
return Qgate

elseif theta != nothing && phi != nothing
Qgate = Qgate(theta=theta,phi=phi)
return Qgate

elseif theta != nothing && lambda != nothing
Qgate = Qgate(theta=theta,lambda=lambda)
return Qgate

elseif phi != nothing && lambda != nothing
Qgate = Qgate(phi=phi,lambda=lambda)
return Qgate

elseif theta != nothing
Qgate = Qgate(theta=theta)
return Qgate

elseif phi != nothing
Qgate = Qgate(phi=phi)
return Qgate

elseif lambda != nothing
Qgate = Qgate(lambda=lambda)
return Qgate

else
return error("Error in Qgate_rot_prep")
end # end if

end # end Qgate_prep

qU_prep (generic function with 1 method)

In [25]:
Ugate = q.P

phase_gate (generic function with 1 method)

In [26]:
Qgate_test = qU_prep(Ugate,lambda=pi/2)

2×2 Matrix{ComplexF64}:
 1.0+0.0im         -0.0-0.0im
 0.0+0.0im  6.12323e-17+1.0im

In [ ]:
#Qgate_test = qU_prep(Ugate,theta=pi/2,phi=pi/2,lambda=pi/2)

In [27]:
Ugate(lambda=pi/2)

2×2 Matrix{ComplexF64}:
 1.0+0.0im         -0.0-0.0im
 0.0+0.0im  6.12323e-17+1.0im

In [28]:
Qgate_test = qU_prep(Ugate,lambda=pi/2)

2×2 Matrix{ComplexF64}:
 1.0+0.0im         -0.0-0.0im
 0.0+0.0im  6.12323e-17+1.0im

In [29]:
# function to choose the corresponding tensor library 
function Op_tensor(Qgate;nqubits::Int64, 
                   q1control=nothing,q2control=nothing,qtarget::Int64,
                   big_endian::Bool=conventions.big_endian)
# get the size of Qgate matrix 
# get the size of Qgate
    n_rows, n_cols = size(Qgate)   
# test if the size of the gate is 2x2
    if n_rows != 2 && n_cols != 2
        error("The size of the gate is not 2x2")
    end 

# once passing the previous check, we can check the size of the gate
# if the size is 2x2, then it is a single qubit gate
# if the size is 4x4, then it is a two qubit gate
# if the size is 8x8, then it is a three qubit gate
# This helps us to determine the tensor library to use   
# in most cases, the one qubit gate is sufficient to determine 
# the tensor library to use with the information about 
# the control qubits and the target qubit 
    # single qubit gate
    # check if the user has provided a control qubit
    if q1control == nothing && q2control == nothing
        # no control qubit1 and qubit2, only target qubit
        # return the qubit gate in the Hilbert space of a 
        # quantum register size 
        Qgate = q_T2D(Qgate, qtarget=qtarget, nqubits=nqubits, 
                      big_endian=big_endian)
        return Qgate
    elseif q2control == nothing
        # only control qubit1 and target qubit
        # return the qubit gate in the Hilbert space of a
        # quantum register size
        Qgate = q_T4D(Qgate, qcontrol=q1control, qtarget=qtarget, 
                        nqubits=nqubits, big_endian=big_endian)
        return Qgate
    end # end if q1control == nothing && q2control == nothing
# now, let's export the gate represented in the Hilbert space of a
# quantum register size
end # end Op_tensor

Op_tensor (generic function with 1 method)

In [30]:
Ugate(lambda=pi/2)

2×2 Matrix{ComplexF64}:
 1.0+0.0im         -0.0-0.0im
 0.0+0.0im  6.12323e-17+1.0im

In [31]:
# check if the user has provided a rotational/phase gate 
# the check is done by checking if the user has provided the
# theta, phi, and lambda parameters
# if so the gate is evaluated and the corresponding matrix is returned
Qgate = qU_prep(Ugate,lambda=pi/2)


2×2 Matrix{ComplexF64}:
 1.0+0.0im         -0.0-0.0im
 0.0+0.0im  6.12323e-17+1.0im

In [32]:
Qgate = q_T2D(Qgate, qtarget=0, nqubits=2, 
                      big_endian=big_endian)

4×4 Matrix{ComplexF64}:
 1.0+0.0im  0.0+0.0im         -0.0-0.0im         -0.0-0.0im
 0.0+0.0im  1.0+0.0im         -0.0-0.0im         -0.0-0.0im
 0.0+0.0im  0.0+0.0im  6.12323e-17+1.0im          0.0+0.0im
 0.0+0.0im  0.0+0.0im          0.0+0.0im  6.12323e-17+1.0im

In [33]:
# test the function Op_tensor
# check if the user has provided a rotational/phase gate 
# the check is done by checking if the user has provided the
# theta, phi, and lambda parameters
# if so the gate is evaluated and the corresponding matrix is returned
gate_tensor = Op_tensor(q.U(lambda=pi/2), nqubits=2, qtarget=0)

4×4 Matrix{ComplexF64}:
 1.0+0.0im  0.0+0.0im         -0.0-0.0im         -0.0-0.0im
 0.0+0.0im  1.0+0.0im         -0.0-0.0im         -0.0-0.0im
 0.0+0.0im  0.0+0.0im  6.12323e-17+1.0im          0.0+0.0im
 0.0+0.0im  0.0+0.0im          0.0+0.0im  6.12323e-17+1.0im

In [52]:
# function to update the table of quantum gates and operations
function qctab_update(;qctab, Qgate, 
    q1control=nothing,q2control=nothing,qtarget::Int64,
    theta=nothing,phi=nothing,lambda=nothing, 
    op_name=nothing,op_mat=nothing,
    big_endian::Bool=conventions.big_endian)

    # get the size of the rows of the table
    n_rows, n_cols = size(qctab)
    qctab1 = qctab
    if n_rows==1
        op_ind = 1
    else
        op_ind = op_ind+1
    end
    # table is 
    #op_tab[1,1:10]=["Op_ind","Op","q1_control","q2_control","q_target", 
    #"theta","phi","lambda","object","op_mat"]
    qctab2 = [op_ind, op_name, q1control, q2control, qtarget, 
        theta, phi, lambda, Qgate, op_mat]
        return vcat(qctab1, qctab2)
end # end qctab_update
    

qctab_update (generic function with 1 method)

In [ ]:
# test updating the table of quantum gates and operations


In [59]:

# Apply a quantum gate to the quantum register
function op_test1(qc, Qgate; q1_control=nothing, q2_control=nothing, 
    q_target::Int64, theta=nothing, phi=nothing, lambda=nothing, 
    err_tol::Float64=conventions.err_tol)
    # Apply a quantum gate to the quantum register
    # qc::quantum register
    # gate::Qgate: quantum gate, 2x2 matrix representing a single qubit gate
    # return: quantum register with the quantum gate applied
    # reading the data from the quantum register qc
    Nqubits = qc.n_qubits
    Nstates = qc.n_dim
    state_vector = qc.state_vector
    
    # first record the name of the function
    Qgate_name = string(Qgate)
    # Evaluate the gate and prepare the gate for the tensor product
    # First, check if the function is a rotational/phase gate 
    # and if so Evaluate it over the theta, phi, and lambda parameters
    # if not, then the function is a single qubit gate
    # check if the user has provided a rotational/phase gate 
    # the check is done by checking if the user has provided the
    # theta, phi, and lambda parameters
    # if so the gate is evaluated and the corresponding matrix is returned
    Qgate = qU_prep(Qgate,theta=theta,phi=phi,lambda=lambda)

    # test the gate 
    # First check if the dimensions of the quantum gate are the same 
    Qgate_dim = size(Qgate)
    if Qgate_dim[1] != Qgate_dim[2]
        error("The quantum gate is not square")
    end # end if
    # check if the quantum gate is unitary
    #if ishermitian(Qgate) == false
    #    error("The quantum gate is not unitary")
    #end # end if
    # check the unitary condition
    UU = Qgate'*Qgate
    II = Matrix(I, Qgate_dim[1], Qgate_dim[2])
    if isapprox(UU, II,rtol=err_tol) == false
        error("The gate is not unitary")
    end
    # check if the dimensions of the quantum gate 
    # ... and the quantum register do match 
    if Qgate_dim[1] != Nstates
        error("The quantum gate and the quantum register do not match")
    end # end if

    gate_tensor = Op_tensor(Qgate, nqubits=Nqubits,q1control=q1_control, 
                            q2control=q2_control,qtarget=q_target,
                            big_endian=big_endian)
    # and the gate is evaluated directly
    # in the Hilbert space
    
    # Apply the quantum gate to the quantum register
    state_vector = gate_tensor * state_vector
    qc.state_vector = state_vector

    # update the matrix representation of the quantum circuit
    qc.qc_matrix = gate_tensor * qc.qc_matrix

    # update the op_table
    qctab = qc.op_table
    #qc.op_table = qctab_update(qctab=qctab, Qgate=Qgate, 
    #q1control=q1_control,q2control=q2_control,qtarget=q_target,
    #theta=theta,phi=phi,lambda=lambda, 
    #op_name=Qgate_name,op_mat= qc.qc_matrix)

    return qc
end # end apply_gate!


op_test1 (generic function with 1 method)

In [56]:
qc = qc_init_test(1)

qc_initstruct1(1, "big-endian", 2, 2, [1.0, 0.0], [0, 1], Any["Op_ind" "Op" … "object" "op_mat"], Bool[1 0; 0 1], true, false, false)

In [45]:
qc.qc_matrix

2×2 Matrix{Bool}:
 1  0
 0  1

In [44]:
q.X

2×2 Matrix{Float64}:
 0.0  1.0
 1.0  0.0

In [60]:
op_test1(qc, q.H, q_target=0)

qc_initstruct1(1, "big-endian", 2, 2, [0.7071067811865474, 0.7071067811865474], [0, 1], Any["Op_ind" "Op" … "object" "op_mat"], [0.7071067811865474 0.7071067811865474; 0.7071067811865474 -0.7071067811865474], true, false, false)

In [49]:
qc.qc_matrix

2×2 Matrix{Float64}:
  0.707107  0.707107
 -0.707107  0.707107

In [ ]:

# Apply a quantum gate to the quantum register
function op_test(qc, Qgate)
    # Apply a quantum gate to the quantum register
    # qc::quantum register
    # gate::Qgate: quantum gate
    # return: quantum register with the quantum gate applied
    Nqubits = qc.n_qubits
    Nstates = qc.n_dim
    state_vector = qc.state_vector
    # First check if the dimensions of the quantum gate are the same 
    Qgate_dim = size(Qgate)
    if Qgate_dim[1] != Qgate_dim[2]
        error("The quantum gate is not square")
    end # end if
    # check if the quantum gate is unitary
    #if ishermitian(Qgate) == false
    #    error("The quantum gate is not unitary")
    #end # end if
    # check the unitary condition
    UU = Qgate'*Qgate
    II = Matrix(I, Qgate_dim[1], Qgate_dim[2])
    if isapprox(UU, II,rtol=err_tol) == false
        error("The gate is not unitary")
    end
    # check if the dimensions of the quantum gate 
    # ... and the quantum register do match 
    if Qgate_dim[1] != Nstates
        error("The quantum gate and the quantum register do not match")
    end # end if
    # Apply the quantum gate to the quantum register
    state_vector = Qgate * state_vector
    qc.state_vector = state_vector
    return qc
end # end apply_gate!


In [ ]:
H = Qgate.H
op_test(qc, Qgate)

In [ ]:
function Unitary(theta)
    qc = qc_init(1, false)
    p = Qgate.P(pi*2*theta)
    op(qc,p)
    return qc
end

In [ ]:
theta = 1/2 + 1/4 + 1/8
unitary = Unitary(theta)

In [ ]:
qc = qc_init(2)

In [ ]:
show_statevector(unitary)

In [ ]:
CtrlU_be = Qgate.ctrl_gate

In [ ]:
function f_test(H_op; nqubits=1, convention=big_endian)
    qc = qc_init(nqubits, convention)
    op(qc, H_op)
    show_statevector(qc)
end

In [ ]:
H_op = Rz_gate1(π/2)

In [ ]:
f_test(H_op; nqubits=1, convention=big_endian)

In [ ]:
f_test(H_op; convention=big_endian, nqubits=1)

In [ ]:
# define a function that takes string as input for the name of the gates
function f_test1(H::String; nqubits=2, convention=big_endian)
    if H == "H"
        H_op = [1 1; 1 -1]/sqrt(2)
    elseif H == "X"
        H_op = [0 1; 1 0]
    elseif H == "Y"
        H_op = [0 -im; im 0]
    end 
    qc = qc_init(nqubits, convention)
    op(qc, H_op)
    show_statevector(qc)
end


In [ ]:
f_test1("H"; nqubits=1, convention=big_endian)

In [ ]:
qc = qc_init(2, big_endian)

# current format of the state vector
H = q.H
H_tensor = q_T2D(H, 0, 2)
qc = qc_init(2, big_endian)
op(qc, H_tensor)




In [ ]:
function op(qc, XX ,qtarget = 0)

    # prepare the quantum gate in the Hilbert space
    if size(H) == (2,2) && qtarget == 0 && qcontrol == nothing 
        H_tensor = q_T2D(H, qtarget, qc.n_qubits)
    H_tensor = q_T2D(H, qtarget, qc.n_qubits)   

  # H_tensor 
    string(XX)

    
    qc.statevector
    qc.table 
    # update the table everytime 

  # table 




In [ ]:

q.

In [ ]:
## create a table 
# data structure 
# converting our simulator to other simulators: TKET, QASM, Qiskit, ...etc. 
# array 
[action gate/operator control1_qubit,   control2_qubit, target_qubit theta     phi     lambda; gate_object
    1     "H"           nothing               nothing         0    nothing  nothing  nothing;    name of function or matrix   
    2     "CX"        0                     nothing         1    nothing  nothing  nothing;
    3     "CX"        1                     nothing         2    nothing  nothing  nothing;
    4      "X"          nothing               nothing         1                             ;
    5      "Y"          nothing               nothing         2                             ;
    6      "Z"          nothing               nothing         3                             ;
    7      "S"          nothing               nothing         4                             ;
   8      "CU"         0                     nothing         2     pi/2      pi/3      pi/4;
    ]
   # n_qubits
    # barrier gate: indentity with the tag plotted on the circuit 1, when drawing the circuit 
    # it should appear in the plot. 
    # **************************
    # convention
    # op: taget qubit, control1 qubit, control2 qubit, ...



    function plot_circuit(qc; nqubits )
        # plot the circuit
        # plot the statevector
        nqubits = qc.n_qubits
        convention = qc.convention
        gate_table = qc.gate_table


    end

    # references 
    # https://typedtables.juliadata.org/stable/man/table/

In [ ]:

H = Qgate.H
X = Qgate.X
Y = Qgate.Y
Z = Qgate.Z
S = Qgate.S

op(qc, Qgate.H(qtarget = 0))



In [ ]:
import Pkg; Pkg.add("TypedTables")
using TypedTables


In [ ]:
`import Pkg; Pkg.add("TypedTables")`
using TypedTables

julia> t = Table(a = [1, 2, 3], b = [2.0, 4.0, 6.0])
Table with 2 columns and 3 rows:
     a  b
   ┌───────
 1 │ 1  2.0
 2 │ 2  4.0
 3 │ 3  6.0

julia> t[1]  # Get first row
(a = 1, b = 2.0)

julia> t.a  # Get column `a`
3-element Array{Int64,1}:
 1
 2
 3